In [2]:
import os
import tarfile

def create_targz_file(source_dir, output_file):
    """创建tar.gz压缩文件。如果文件已存在，则跳过打包。"""
    if not os.path.exists(output_file):
        with tarfile.open(output_file, 'w:gz') as tar:
            tar.add(source_dir, arcname=os.path.basename(source_dir))
        print(f"已成功创建tar.gz文件: {output_file}")
    else:
        print(f"Tar.gz文件已经存在: {output_file}, 将跳过创建.")

def format_file_dict(download_type, author_name, cloud_directory, file_name, parent_directory_name, son_path, metadata=None):
    """格式化文件字典"""
    file_dict = (
        "{\n"
        "  'downloadType': '" + download_type + "',\n"
        "  'url': '" + f"{author_name}/{cloud_directory}/{file_name}" + "',\n"
        "  'parentPath': '" + parent_directory_name + "',\n"
        "  'sonPath': '" + son_path + "',\n"
        "  'fileName': '" + file_name + "',\n"
    )

    if metadata and download_type == "cg_targz":
        file_dict += "  'metadata': [\n" + ',\n'.join(metadata) + "\n  ]\n"

    file_dict += "}"
    return file_dict

def format_metadata_entry(son_path, file_name):
    """格式化metadata条目"""
    return (
        "    {\n"
        "      'sonPath': '" + (son_path if son_path.startswith('/') else '/' + son_path) + "',\n"
        "      'fileName': '" + file_name + "'\n"
        "    }"
    )

def generate_code(download_type, parent_directory, model_category, model_name, author_name, cloud_directory, parent_directory_name, relative_directory):
    """生成代码，并格式化输出信息，检查重复文件"""
    files_list = []
    metadata_list = []
    file_names = set()  # 用于存储文件名以检查重复

    # 处理cg_targz类别
    if download_type == "cg_targz":
        targz_file_name = os.path.basename(parent_directory) + '.tar.gz'
        create_targz_file(parent_directory, targz_file_name)

    # 遍历文件目录
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            full_path = os.path.join(root, file)
            son_path = os.path.relpath(os.path.dirname(full_path), relative_directory)
            if not son_path.startswith('/'):
                son_path = '/' + son_path

            # 检查文件名重复
            if download_type == "cg":
                file_key = (os.path.splitext(file)[0], os.path.splitext(file)[1])  # 文件名和后缀
                if file_key in file_names:
                    print(f"重复的文件: {file} 在路径: {son_path}")
                else:
                    file_names.add(file_key)
                files_list.append(format_file_dict(download_type, author_name, cloud_directory, file, parent_directory_name, son_path))
            elif download_type == "cg_targz":
                metadata_list.append(format_metadata_entry(son_path, file))

    if download_type == "cg_targz":
        files_list.append(format_file_dict(download_type, author_name, cloud_directory, targz_file_name, parent_directory_name, "/", metadata_list))

    # 将列表转换为字符串
    files_list_str = ',\n'.join(files_list)

    # 构建代码字符串
    code = (
        "manager.add_mod_childs("
        + model_category + ", "
        "'" + model_name + "', "
        "[\n" + files_list_str + "\n  ]\n"
        ")"
    )

    return code

# 示例用法
parent_directory = "/root/autodl-tmp/models/interrogators" # 扫描的文件目录
model_category = "a6" # 所属类别
model_name = "TAG插件反推依赖模型包" # 展示的模型名字
author_name = "xiaolxl" # cg模型站上的作者名字
cloud_directory = "tag_test_mods" # cg模型站上的模型名字
parent_directory_name = "interrogators_dir" # 所属父目录类别
relative_directory = "/root/autodl-tmp/models/interrogators" # 相对于哪一个父目录进行子目录设置

download_type = "cg_targz" #cg, cg_targz

# 生成代码
generated_code = generate_code(download_type, parent_directory, model_category, model_name, author_name, cloud_directory, parent_directory_name, relative_directory)
print(generated_code)


manager.add_mod_childs("插件所需依赖/模型", "AnimateDifV3的SparseControlNet模型", [
    {
        "downloadType": "cg",
        "url": "xiaolxl/AnimateDiff-A1111-ControlNetMod/mm_sd15_v3_sparsectrl_rgb.safetensors",
        "parentPath": "controlnet_dir",
        "sonPath": "/",
        "fileName": "mm_sd15_v3_sparsectrl_rgb.safetensors"
    },
    {
        "downloadType": "cg",
        "url": "xiaolxl/AnimateDiff-A1111-ControlNetMod/mm_sd15_v3_sparsectrl_scribble.safetensors",
        "parentPath": "controlnet_dir",
        "sonPath": "/",
        "fileName": "mm_sd15_v3_sparsectrl_scribble.safetensors"
    },
])
